In [20]:
#Take Care of the Imports
import pandas as pd
import numpy as np
import torch
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
import torch
import torch.nn as nn
from torch.nn import Linear
from torch.utils.data import Dataset, DataLoader

2.1.1


In [2]:
'''
Set Parent Directories For:

sensor_geometry.csv
batch_##.parquet
train_meta.parquet

respectively

'''

#Ben
# sensor_geom_path = '/opt/app/data/erdos-data/'
# batch_dir = '/opt/app/data/erdos-data/train/'
# meta_dir = '/opt/app/data/erdos-data/'
sensor_geom_dir = "D:/jupyter/erdos-data/"
batch_dir = "D:/jupyter/erdos-data/train/"
meta_dir = "D:/jupyter/erdos-data/"

#Chinmaya
#sensor_geom_dir =
#batch_dir =
#meta_dir =

##Katja
#sensor_geom_dir =
#batch_dir =
#meta_dir =

##Chris
#sensor_geom_dir =
#batch_dir =
#meta_dir =

##Lukas
#sensor_geom_dir =
#batch_dir =
#meta_dir =

In [3]:
#Load training metadata if not already loaded
meta_path=meta_dir+"train_meta.parquet"

try: meta
except NameError: meta=pd.read_parquet(meta_path)

In [22]:
# Set Default Device to CUDA or CPU
#to be called in .to(device) to ensure all pytorch Tensors are on the same device
device = (
    "cuda:0"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
torch.set_default_device('cuda:0')
print(f"Using {device} device")

Using cuda:0 device


In [12]:
# Class for a dataset generated from a dataframe and data from the sensor geometry file
class NeutrinoDataset(Dataset):
    def __init__(self, batch_filename, sensor_file_name, batch_id):
        self.sensor_geom = pd.read_csv(sensor_file_name)
        self.vals_df = meta[meta.batch_id==batch_id] 
        self.dataframe = pd.read_parquet(batch_filename)
        sensor_loc = np.array(self.sensor_geom.iloc[:])[:, 1:]
        self.num_features = 5160*3
        self.num_events = self.dataframe.index.nunique()
        self.unique_indices = np.unique(self.dataframe.index)
        
    def __len__(self):
        return self.num_events
    
    # Replaces sensor ID with sensor coordinates 
    def __getitem__(self, i):
        df = self.dataframe
        sg = self.sensor_geom
        meta_vals = np.array(
            self.vals_df.loc[self.vals_df['event_id'] == df.index[i]])[0].astype(float)
        
        pulse_array = np.array(df.loc[df.index[i]])
        pulse_array_sensors = np.concatenate((np.expand_dims(np.arange(5160), axis=1), np.zeros([5160, 3])), 1)

        for pulse in pulse_array:
            if(pulse_array_sensors[pulse[0]][1] == 0):
                pulse_array_sensors[pulse[0]][1] = pulse[1] - meta_vals[2] # first time
            else:
                # possible last time, will be the last time for the actual last one
                pulse_array_sensors[pulse[0]][2] = pulse[1] - meta_vals[2]
            # Add charge
            pulse_array_sensors[pulse[0]][3] += pulse[2]
        
        flattened_pulse = (pulse_array_sensors[:, 1:]).flatten()
        # print(flattened_pulse.shape)
                
        return (torch.from_numpy(flattened_pulse), 
                                 torch.from_numpy(meta_vals[-2:]))
    
    # Finds the first event with multiple pulses at the same sensors
    # here we ask for at least num_min_total_repeats repetitions
    def get_multi_pulse_event(self, num_min_total_repeats):
        for i in range(self.num_events):
            pulses = np.array(df.loc[unique_indices[i]])
            if(pulses[:,0].shape[0] - np.unique(pulses[:,0]).shape[0] >= num_min_total_repeats):
                return self.unique_indices[i]
            
    # Finds all events in a range with multiple pulses at the same sensors
    # here we ask for at least num_min_total_repeats repetitions
    def get_multi_pulse_events(self, num_min_total_repeats, start_index, end_index):
        list_multi_pulse = []
        for i in range(start_index, min(self.num_events, end_index)):
            pulses = np.array(df.loc[unique_indices[i]])
            if(pulses[:,0].shape[0] - np.unique(pulses[:,0]).shape[0] >= num_min_total_repeats):
                list_multi_pulse.append(self.unique_indices[i])
        return list_multi_pulse


In [13]:
# Define the Neural Net Class
class NNPredictor(torch.nn.Module):
    def __init__(self, use_activation = True):
        super().__init__()
        # torch.manual_seed(1234)
        self.layers = nn.ModuleList()
        self.layer_norms = nn.ModuleList()
        self.use_activation = use_activation
        
        self.layers.append(nn.Linear(dataset.num_features, 100, dtype=float))
        self.layers.append(nn.Linear(100, 50, dtype=float))
        self.layers.append(nn.Linear(50, 10, dtype=float))
        self.classifier = (nn.Linear(10,2, dtype=float))

    def forward(self, x):
        new_x = x.to(device)
        if(self.use_activation):
            for layer in self.layers:
                #print(layer, new_x.dtype)
                new_x = layer(new_x)
                new_x = nn.ReLU()(new_x)
        else:
            for layer in self.layers:
                new_x = layer(new_x)
        
        # Apply a final (linear) classifier.

        return self.classifier(new_x)

In [14]:
#define our custom loss class
class custom_MAE(nn.Module):
    def __init__(self):
        super(custom_MAE, self).__init__();

    def forward(self, predictions, target):
        loss_value = self.angular_dist_score(predictions, target).to(device)
        return loss_value
    
    def angular_dist_score(self, predictions, true):
#     '''
#     calculate the MAE of the angular distance between two directions.
#     The two vectors are first converted to cartesian unit vectors,
#     and then their scalar product is computed, which is equal to
#     the cosine of the angle between the two vectors. The inverse 
#     cosine (arccos) thereof is then the angle between the two input vectors
    
#     Parameters:
#     -----------
    
#     az_true : float (or array thereof)
#         true azimuth value(s) in radian
#     zen_true : float (or array thereof)
#         true zenith value(s) in radian
#     az_pred : float (or array thereof)
#         predicted azimuth value(s) in radian
#     zen_pred : float (or array thereof)
#         predicted zenith value(s) in radian
    
#     Returns:
#     --------
    
#     dist : float
#         mean over the angular distance(s) in radian
#     '''
        az_true=true[0].to(device)
        zen_true=true[1].to(device)
        az_pred=predictions[0].to(device)
        zen_pred=predictions[1].to(device)
    
        if not (torch.all(torch.isfinite(az_true)) and
                torch.all(torch.isfinite(zen_true)) and
                torch.all(torch.isfinite(az_pred)) and
                torch.all(torch.isfinite(zen_pred))):
            raise ValueError("All arguments must be finite")
    
        # pre-compute all sine and cosine values
        sa1 = torch.sin(az_true).to(device).to(device)
        ca1 = torch.cos(az_true).to(device)
        sz1 = torch.sin(zen_true).to(device)
        cz1 = torch.cos(zen_true).to(device)
    
        sa2 = torch.sin(az_pred).to(device)
        ca2 = torch.cos(az_pred).to(device)
        sz2 = torch.sin(zen_pred).to(device)
        cz2 = torch.cos(zen_pred).to(device)
    
        # scalar product of the two cartesian vectors (x = sz*ca, y = sz*sa, z = cz)
        scalar_prod = sz1*sz2*(ca1*ca2 + sa1*sa2) + (cz1*cz2)
    
        # scalar product of two unit vectors is always between -1 and 1, this is against nummerical instability
        # that might otherwise occure from the finite precision of the sine and cosine functions
        scalar_prod =  torch.clip(scalar_prod, -1, 1)
    
        # convert back to an angle (in radian)
        return torch.mean(torch.abs(torch.arccos(scalar_prod))).to(device)

In [15]:
#Define the Training Loop
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # print(X)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 10 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            


In [18]:
#Initialize NN

#Pick training batch
batch_id=104

#Set NN Parameters
batch_size=512
learning_rate = 1e-9
loss_fn = custom_MAE()


#Generate Full Paths
sensor_geom_path=sensor_geom_dir+"sensor_geometry.csv"
batch_path=batch_dir+"batch_"+str(batch_id)+".parquet"

#initialize Model, Data, DataLoader, and Optimizer for the training loop
dataset = NeutrinoDataset(batch_path, sensor_geom_path, batch_id)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0, generator=torch.Generator(device=device))

model = NNPredictor()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [19]:
train_loop(dataloader, model, loss_fn, optimizer)

loss: 0.734040  [  512/200000]
loss: 1.510379  [ 5632/200000]
loss: 1.586608  [10752/200000]
loss: 1.578400  [15872/200000]
loss: 0.775274  [20992/200000]
loss: 1.711835  [26112/200000]
loss: 1.930576  [31232/200000]
loss: 1.484451  [36352/200000]
loss: 1.346926  [41472/200000]
loss: 1.611818  [46592/200000]
loss: 2.182592  [51712/200000]
loss: 1.934145  [56832/200000]
loss: 1.369191  [61952/200000]
loss: 1.588051  [67072/200000]
loss: 1.475768  [72192/200000]
loss: 1.317761  [77312/200000]
loss: 1.735174  [82432/200000]
loss: 1.446872  [87552/200000]
loss: 0.491263  [92672/200000]
loss: 1.067228  [97792/200000]
loss: 2.086958  [102912/200000]
loss: 0.976991  [108032/200000]
loss: 1.301224  [113152/200000]
loss: 2.393096  [118272/200000]
loss: 1.310644  [123392/200000]
loss: 1.204270  [128512/200000]
loss: 0.727586  [133632/200000]
loss: 0.307125  [138752/200000]
loss: 0.648113  [143872/200000]
loss: 2.692681  [148992/200000]
loss: 1.316685  [154112/200000]
loss: 1.678863  [159232/2000

In [ ]:
# i=0
# meta_vals = np.array(batch_104_vals_df.loc[batch_104_vals_df['event_id'] 
#                                            == df.index[0]])[0].astype(float)
        
# pulse_array = np.array(df.loc[df.index[i]])
# pulse_array_sensors = np.concatenate((np.expand_dims(np.arange(5160), axis=1), np.zeros([5160, 3])), 1)

# for pulse in pulse_array:
#     if(pulse_array_sensors[pulse[0]][1] == 0):
#         pulse_array_sensors[pulse[0]][1] = pulse[1] - meta_vals[2] # first time
#     else:
#         # possible last time, will be the last time for the actual last one
#         pulse_array_sensors[pulse[0]][2] = pulse[1] - meta_vals[2]
#     # Add charge
#     pulse_array_sensors[pulse[0]][3] += pulse[2]

# print(torch.from_numpy(np.concatenate(
#     (np.ndarray.flatten(pulse_array_sensors[:, 1:]), meta_vals[-2:]))))

In [ ]:
test_batch_id=1
test_batch_path=batch_path="D:/jupyter/erdos-data/train/batch_"+str(test_batch_id)+".parquet"
test_data=NeutrinoDataset(test_batch_path, sensor_geom_path, test_batch_id)
test_dataloader=DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0, generator=torch.Generator(device=device))

loss_total = 0
num = 0
with torch.no_grad():
    for batch, (X, y) in enumerate(test_dataloader):
        # print(X)
        # Compute prediction and loss
        pred = model(X)
        loss_total += loss_fn.angular_dist_score(pred,y)
        num +=1
        print(loss_total/num)


tensor(1.2100, device='cuda:0', dtype=torch.float64)
tensor(1.1646, device='cuda:0', dtype=torch.float64)
tensor(1.3265, device='cuda:0', dtype=torch.float64)
tensor(1.4339, device='cuda:0', dtype=torch.float64)
tensor(1.5156, device='cuda:0', dtype=torch.float64)
tensor(1.6709, device='cuda:0', dtype=torch.float64)
tensor(1.5125, device='cuda:0', dtype=torch.float64)
tensor(1.4238, device='cuda:0', dtype=torch.float64)
tensor(1.3852, device='cuda:0', dtype=torch.float64)
tensor(1.3715, device='cuda:0', dtype=torch.float64)
tensor(1.3827, device='cuda:0', dtype=torch.float64)
tensor(1.2968, device='cuda:0', dtype=torch.float64)
tensor(1.2865, device='cuda:0', dtype=torch.float64)
tensor(1.3138, device='cuda:0', dtype=torch.float64)
tensor(1.2570, device='cuda:0', dtype=torch.float64)
tensor(1.2261, device='cuda:0', dtype=torch.float64)
tensor(1.2685, device='cuda:0', dtype=torch.float64)
tensor(1.2357, device='cuda:0', dtype=torch.float64)
tensor(1.2202, device='cuda:0', dtype=torch.fl